In [0]:
import numpy as np
import pandas as pd
from imblearn.over_sampling import RandomOverSampler as ROS
from imblearn.over_sampling import SMOTE as SMO

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
train =pd.read_csv('/content/gdrive/My Drive/ML/Training_x.csv',sep='\t', encoding='utf-8')
test = pd.read_csv('/content/gdrive/My Drive/ML/Testing_x.csv',sep='\t', encoding='utf-8')


In [0]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1467214 entries, 0 to 1467213
Data columns (total 42 columns):
Unnamed: 0         1467214 non-null int64
Quarter            1467214 non-null float64
Month              1467214 non-null int64
DayofMonth         1467214 non-null int64
DayOfWeek          1467214 non-null float64
FlightNum          1467214 non-null float64
Origin             1467214 non-null object
DestAirportID      1467214 non-null float64
Dest               1467214 non-null object
CRSDepTime         1467214 non-null int64
DepTime            1467214 non-null float64
DepDelay           1467214 non-null float64
DepDelayMinutes    1467214 non-null float64
DepDel15           1467214 non-null float64
CRSArrTime         1467214 non-null float64
ArrTime            1467214 non-null float64
ArrDelayMinutes    1467214 non-null float64
ArrDel15           1467214 non-null float64
year               1467214 non-null int64
DewPointC          1467214 non-null float64
DewPointF          

In [0]:
data = pd.concat([train,test],axis = 0)
data = data.reset_index()
data = data.drop(columns = ['index'])

In [0]:
data['isDelay'] = 0

def func(row):
    if row['DepDel15']>0:
        return 1
    else:
        return 0

data['isDelay'] = data.apply(func, axis=1)


In [0]:
len(data)

1834018

In [0]:
data['isDelay'].plot(kind='pie', subplots=True, figsize=(8, 8))


In [0]:
x_tr = ['Quarter', 'Month', 'DayofMonth', 'DayOfWeek','CRSDepTime',
       'FlightNum', 'Origin', 'DestAirportID', 'Dest','year',
       'DewPointC', 'DewPointF', 'FeelsLikeC', 'FeelsLikeF', 'HeatIndexC',
       'HeatIndexF', 'WindChillC', 'WindChillF', 'WindGustKmph',
       'WindGustMiles', 'cloudcover', 'humidity', 'precipMM', 'pressure',
       'tempC', 'tempF', 'time', 'visibility', 'weatherCode', 'winddir16Point',
       'winddirDegree', 'windspeedKmph', 'windspeedMiles']
X= data[x_tr]
x_val = data[x_tr].iloc[:,:].values

In [0]:
x_val[0]

In [0]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
enc  = LabelEncoder()
enc.fit(x_val[:,8].reshape(-1, 1) )
c1=['d1']
temp = pd.DataFrame(enc.transform(x_val[:,6]),columns = c1)
X = pd.concat([X,temp],axis=1)
X = X.drop(columns = ['Origin'])
c2=['a1']
temp = pd.DataFrame(enc.transform(x_val[:,8]),columns = c2)
X = pd.concat([X,temp],axis=1)
X = X.drop(columns = ['Dest'])
X = X.drop(columns = ['winddir16Point'])
X = X.drop(columns = ['DewPointF','FeelsLikeC','FeelsLikeF','HeatIndexF','WindChillF','WindGustMiles','tempF','windspeedMiles','Quarter'])
x_val = X.iloc[:,:].values
y = data['isDelay']
y_val  = y.iloc[:].values

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [0]:
sampler = SMO()

In [0]:
x_val,y_val =  sampler.fit_resample(x_val,y_val)

In [0]:
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(x_val, y_val, test_size=0.33, random_state=42)

In [0]:
del X
del y
del train
del test
del data

In [0]:
from sklearn.ensemble import RandomForestClassifier
classifier2 = RandomForestClassifier(n_estimators=10)
classifier2.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [0]:
print(classifier2.score(X_test, y_test, sample_weight=None))
from sklearn.metrics import classification_report
y_pred1 = classifier2.predict(X_test)
print(classification_report(y_true=y_test,y_pred = y_pred1))

0.8514230434122035
              precision    recall  f1-score   support

           0       0.82      0.90      0.86    483269
           1       0.89      0.80      0.84    484270

    accuracy                           0.85    967539
   macro avg       0.85      0.85      0.85    967539
weighted avg       0.85      0.85      0.85    967539



In [0]:

from sklearn.ensemble import GradientBoostingClassifier as GBC

In [0]:
classifier3 = GBC(n_estimators= 30)
classifier3.fit(X_train,y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=30,
                           n_iter_no_change=None, presort='auto',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [0]:
print(classifier3.score(X_test, y_test, sample_weight=None))
from sklearn.metrics import classification_report
y_pred1 = classifier3.predict(X_test)
print(classification_report(y_true=y_test,y_pred = y_pred1))

0.8295923988593742
              precision    recall  f1-score   support

           0       0.77      0.94      0.85    483269
           1       0.93      0.72      0.81    484270

    accuracy                           0.83    967539
   macro avg       0.85      0.83      0.83    967539
weighted avg       0.85      0.83      0.83    967539



In [0]:
from sklearn.ensemble import ExtraTreesClassifier as ETC
classifier4 = ETC(n_estimators = 30)
classifier4.fit(X_train,y_train)

ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
                     max_depth=None, max_features='auto', max_leaf_nodes=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=30, n_jobs=None,
                     oob_score=False, random_state=None, verbose=0,
                     warm_start=False)

In [0]:
print(classifier4.score(X_test, y_test, sample_weight=None))
from sklearn.metrics import classification_report
y_pred1 = classifier4.predict(X_test)
print(classification_report(y_true=y_test,y_pred = y_pred1))

0.8406493174952121
              precision    recall  f1-score   support

           0       0.83      0.86      0.84    483269
           1       0.85      0.82      0.84    484270

    accuracy                           0.84    967539
   macro avg       0.84      0.84      0.84    967539
weighted avg       0.84      0.84      0.84    967539



In [0]:
from sklearn import svm
classifier5 = svm.SVC(gamma='scale')
classifier5.fit(X_res,y_res)

In [0]:
print(classifier5.score(x_test_val, y_tst, sample_weight=None))
from sklearn.metrics import classification_report
y_pred1 = classifier5.predict(x_test_val)
print(classification_report(y_true=y_tst,y_pred = y_pred1))

In [0]:
from xgboost import XGBClassifier
model = XGBClassifier(max_depth  = 8, base_score = 0.5,learning_rate = 0.2)
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.2, max_delta_step=0, max_depth=8,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [0]:
from sklearn.metrics import accuracy_score
print(model.score(X_test, y_test, sample_weight=None))
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
from sklearn.metrics import classification_report
y_pred1 = model.predict(X_test)
print(classification_report(y_true=y_test,y_pred = y_pred1))

0.8317762901547121
Accuracy: 83.18%
              precision    recall  f1-score   support

           0       0.77      0.94      0.85    483269
           1       0.93      0.72      0.81    484270

    accuracy                           0.83    967539
   macro avg       0.85      0.83      0.83    967539
weighted avg       0.85      0.83      0.83    967539

